In [98]:
import sys
import os
sys.path.append(os.path.abspath(".."))

# for numeric calculations
import numpy as np

# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Import the DstApi utility class
from dstapi.dstapi import DstApi

In [99]:
# Initialize the class with the target table
data_table = DstApi('STRAFNA6')

# Get the table summary
data_table.tablesummary(language='da')

Table STRAFNA6: Skyldige personer efter område, køn, alder, national oprindelse og tid
Last update: 2024-12-09T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,OMRÅDE,105,000,Hele landet,851,Aalborg,False
1,KØN,2,M,Mænd,K,Kvinder,False
2,ALDER,4,TOT,Alder i alt,50-79,50-79 år,False
3,HERKOMST1,2,1,Personer med dansk oprindelse,6,Personer med udenlandsk oprindelse,False
4,Tid,19,2005,2005,2023,2023,True


In [100]:
# Start by constructing a basic dictionary
params = data_table.define_base_params(language='da')
params['format'] = 'csv'

params

{'table': 'strafna6',
 'format': 'csv',
 'lang': 'da',
 'variables': [{'code': 'OMRÅDE', 'values': ['*']},
  {'code': 'KØN', 'values': ['*']},
  {'code': 'ALDER', 'values': ['*']},
  {'code': 'HERKOMST1', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

In [ ]:
# Variable exclusion rules
exclusions = {
    'OMRÅDE': ['Hele landet'],
    'ALDER': ['Alder i alt'],
    '*': ['I alt']  # Will be applied to all variables in addition to specific ones
}

In [102]:
for var in params['variables']:
    # Extract all possible values for this column
    levels = data_table.variable_levels(var['code'], language='da')

    # Gather exclusions: specific + global
    exclude_texts = set(exclusions.get(var['code'], [])) | set(exclusions.get('*', []))

    # Exclude specified text values
    levels = levels[~levels['text'].isin(exclude_texts)]

    # Assign the filtered id values
    var['values'] = levels['id'].tolist()

In [103]:
params

{'table': 'strafna6',
 'format': 'csv',
 'lang': 'da',
 'variables': [{'code': 'OMRÅDE',
   'values': ['084',
    '101',
    '147',
    '155',
    '185',
    '165',
    '151',
    '153',
    '157',
    '159',
    '161',
    '163',
    '167',
    '169',
    '183',
    '173',
    '175',
    '187',
    '201',
    '240',
    '210',
    '250',
    '190',
    '270',
    '260',
    '217',
    '219',
    '223',
    '230',
    '400',
    '411',
    '085',
    '253',
    '259',
    '350',
    '265',
    '269',
    '320',
    '376',
    '316',
    '326',
    '360',
    '370',
    '306',
    '329',
    '330',
    '340',
    '336',
    '390',
    '083',
    '420',
    '430',
    '440',
    '482',
    '410',
    '480',
    '450',
    '461',
    '479',
    '492',
    '530',
    '561',
    '563',
    '607',
    '510',
    '621',
    '540',
    '550',
    '573',
    '575',
    '630',
    '580',
    '082',
    '710',
    '766',
    '615',
    '707',
    '727',
    '730',
    '741',
    '740',
    '746',

In [104]:
# Fetch all the data from the table with the specified params
df = data_table.get_data(params=params)

In [105]:
df.shape

(23712, 6)

In [106]:
df.sample(5)

,OMRÅDE,KØN,ALDER,HERKOMST1,TID,INDHOLD
15003,Kolding,Kvinder,30-49 år,Personer med udenlandsk oprindelse,2017,32
20369,Skive,Mænd,50-79 år,Personer med dansk oprindelse,2006,208
18304,Silkeborg,Mænd,30-49 år,Personer med udenlandsk oprindelse,2012,85
13683,Billund,Mænd,15-29 år,Personer med dansk oprindelse,2008,167
23336,Vesthimmerlands,Mænd,50-79 år,Personer med dansk oprindelse,2009,148


In [107]:
df.to_csv('../Data/STRAFNA6/CleanedSTRAFNA6.csv', index=False)

In [108]:
# Select all the columns that are not numeric
data_column_category = df.select_dtypes(exclude=[np.number]).columns

In [109]:
# Select all the columns that are not numeric
data_column_category = df.select_dtypes(exclude=[np.number]).columns
# View column categories
data_column_category

Index(['OMRÅDE', 'KØN', 'ALDER', 'HERKOMST1'], dtype='object')

In [110]:
# Get a sample from the datafrane with only the categorical columns
df[data_column_category].sample(5)

,OMRÅDE,KØN,ALDER,HERKOMST1
13669,Ærø,Kvinder,50-79 år,Personer med udenlandsk oprindelse
5303,Gribskov,Mænd,30-49 år,Personer med udenlandsk oprindelse
21197,Region Nordjylland,Kvinder,50-79 år,Personer med udenlandsk oprindelse
13065,Odense,Mænd,30-49 år,Personer med udenlandsk oprindelse
20433,Skive,Kvinder,15-29 år,Personer med udenlandsk oprindelse


In [111]:
label_encoder = LabelEncoder()

In [112]:
# Make a copy of the DataFrame
df_label_encoded = df.copy()

In [113]:
# Performing label encoding
for i in data_column_category:
    df_label_encoded[i] = label_encoder.fit_transform(df_label_encoded[i])

In [114]:
# View the label encoded data
print("Label Encoded Data: ")
df_label_encoded.sample(5)

Label Encoded Data: 


,OMRÅDE,KØN,ALDER,HERKOMST1,TID,INDHOLD
2192,26,0,0,1,2012,32
19471,40,1,2,0,2020,411
512,21,1,1,0,2023,446
4784,19,0,2,1,2020,7
19715,45,1,2,1,2017,20


In [115]:
df_label_encoded.to_csv('../Data/STRAFNA6/EncodedSTRAFNA6.csv', index=False)